# Cuckoo Filter 

### Introduction and Post Summary
Probabilistic data structures are useful in streaming settings and other applications with significant memory requirements. In streaming scenarios, one often wants to test whether an item has been encountered before. For example, imagine an application that consumes tweets from the twitter api. A natural query might be whether a tweet has been encountered before. With hundreds of millions of tweets published daily, storing all tweets in a [set](https://en.wikibooks.org/wiki/Data_Structures/Sets) will incur significant space. Probabilistic data structures are designed to answer such queries in a space efficient manner while potentially sacrificing accuracy. Here, we explore a [recently](https://www.cs.cmu.edu/~dga/papers/cuckoo-conext2014.pdf) introduced probabilistic data structure: the cuckoo filter. The cuckoo filter was designed as an improvement upon the standard bloom filter. In this post, we provide a python implementation of the cuckoo filter, and show comparisons to a counting bloom filter; a bloom filter variant that allows for dynamic deletions.

### Bloom filter

At Fast Forward Labs, bloom filters and other probabilistic data structures were explored in detail in the report, "Probabilistic Methods for Real-time Streams." [Bloom filters](https://en.wikipedia.org/wiki/Bloom_filter) leverage hash functions to encode items compactly in an array. Encoded values derived from hashing an item to be inserted into a bloom filter are then used as indices of an array. Consequently, inserting an item involves setting the indices derived from hashing that item. It is possible for multiple items to hash to the same set of indices; hence, membership queries can result in false positives. Traditional bloom filters do not support deletions, since hashing is lossy and irreversible; hence deletions require the entire filter to be rebuilt. Variants of the bloom filter that allow for deletions have however been introduced. One popular variant we consider here is the counting bloom filter.

### Cuckoo filter

The cuckoo filter is designed to provide capabilities similar to the counting bloom filter. The cuckoo filter consists of a [cuckoo hash table](http://web.stanford.edu/class/archive/cs/cs166/cs166.1146/lectures/13/Small13.pdf) that stores the 'fingerprints' of items inserted. The fingerprint of an item is a reduced bit string derived from the hash of that item. A cuckoo hash table "consists of an array of buckets" where an item to be inserted is mapped to two possible buckets based on two hash functions. Each bucket can be configured to store variable number of fingerprints. Typically, a cuckoo filter is identified  by the its fingerprint size and bucket size. For example, a (2,4)
cuckoo filter stores 2 bit length fingerprints and each bucket in the cuckoo hash table can store up to 4 fingerprints. Following the above [paper](https://www.cs.cmu.edu/~dga/papers/cuckoo-conext2014.pdf), we implemented the cuckoo filter in python. Below, we initialize an example cuckoo filter and test simple inserts and deletions. We also implement a counting bloom filter as well in order to make performance comparisons. Below, we initialize both data structures. 

In [54]:
from cuckoofilter import CuckooFilter
c_filter = CuckooFilter(10000, 2) #specify capacity and fingerprint size

In [55]:
c_filter.insert("James")
print("James" in c_filter)

c_filter.remove("James")
print("James" in c_filter)

True
False


In [56]:
from cuckoofilter import CountingBloomFilter
b_filter = CountingBloomFilter(10000) #specify the capacity of a counting bloom filter

b_filter.add("James")
print("James" in b_filter)

b_filter.remove("James")
print("James" in b_filter)

True
False


The cuckoo filter supports three key operations: insert, delete, and lookup. Of all these operations, the insert operation is most involved. 
to insert an item into the cuckoo filter, one derives two possible indices from the item based on hashing the item and its fingerprint. On obtaining these indices, one then inserts the item fingerprint into one of the 
above buckets. As the cuckoo hash table begins to fill up, one 
can encounter a situation where the two possible indices where an 
item can be inserted has been filled, in this case, inserted 
items can then be swapped into other buckets to free up space for 
a new item. As the cuckoo hash table fills up to capacity, more 
items would need to be swapped. To prevent infinite swapping when 
the cuckoo table fills up, one can limit the swapping to be performed. This number is typically between 400 and 500. See cuckoofilter.py file in the repo for more implementation details.

### Inserting into a cuckoo filter

Here, we now provide code details and further explanations on the insert interface of the cuckoo filter. 

In [57]:
#example function to demonstrate how to insert into a cuckoo filter. 

import mmh3

"""
To insert an item into a cuckoo table. 

hash the item to produce two indices for inserting the item. 
The function obtain indices from item demonstrates how to 
derive two indices from a string item. 
    1) hash the item to produce an integer 
        --map this integer to an index in the cuckoo table using modulo
        --this is the first index
    
    2) take the first N bits of the hash of the item as the fingerprint. 
    
    3) hash the N-bit fingerprint
        --map this integer to an index in the cuckoo table. 
        
    4) second index -> first_index xor index derived from hash(fingerprint)
    
    return both indices. 
    
    
Note: One the table starts to get full, then one would need to move 
items around as indicated in the insert pseudocode in the referenced 
paper. We skip this part here for the sake of clarity. See the repo of a 
complete implementation of the insert functionality. 
        

"""

def obtain_indices_from_item(item_to_insert, fingerprint_size, capacity):
    
    #hash the string item 
    hash_value = mmh3.hash_bytes(item_to_insert)
    
    #subset the hash to a fingerprint size
    fingerprint = hash_value[:fingerprint_size]
    
    #derive the index
    index_1 = int.from_bytes(hash_value, byteorder="big")
    index_1 = index_1 % capacity
    
    #derive the index from the fingerprint
    hashed_fingerprint = mmh3.hash_bytes(fingerprint)
    finger_print_index = int.from_bytes(hashed_fingerprint, byteorder="big")
    finger_print_index = finger_print_index % capacity
    
    #second index -> first_index xor index derived from hash(fingerprint)
    index_2 = index_1 ^ finger_print_index
    index_2 = index_2 % capacity
    
    return index_1, index_2, fingerprint

def insert_into_table(table, index_1, index_2, bucket_capacity):
    #now insert item into the table
    if len(table[index_1]) < bucket_capacity:
        table[index_1].append(fp)
        return table, index_1

    if len(table[index_1]) < bucket_capacity:
        table[index_2].append(fp)
        return table, index_2

    #Move items to other positions in the table. 
    
    return "Unable to Insert Item"    

In [58]:
#let's create a crude cuckoo hashtable
capacity = 10 #capacity of our cuckoo hashtable
bucket_capacity = 4
table = [[] for _ in range(capacity)]

In [59]:
#obtain possibe indices
index_1, index_2, fp = obtain_indices_from_item("James",\
                                            2, 10)

In [60]:
table, _ = insert_into_table(table, index_1, index_2, bucket_capacity)

In [61]:
table

[[], [], [], [], [], [], [], [], [], [b'\xc0\n']]

## Bench marking against counting bloom filter

Now we explore the false positive behavior of the counting bloom filter and the cuckoo filter. A critical metric for probabilistic data structures like the bloom and cuckoo filters is the false positive rate. The false positive rate typically depends on the capacity and memory allocation assigned during the creation of a filter. A key advantage of the cuckoo filter is that, for similar space and item numbers, the cuckoo filter provides lower false positive rates than the bloom filter. In particular, if a false positive rate less than 3 percent is desired, the cuckoo filter the cuckoo filter uses less space. First we demonstrate and example of how to estimate the false positive rate of both filters. 

#### Calculating False Positive Rate

In [62]:
#specify number of items to insert into the filters
max_items = 5000

num_inserted = 0
for i in range(max_items):
    c_filter.insert(str(i))
    b_filter.add(str(i))
    num_inserted += 1

#test to make sure all the inserted numbers are present in the filters
for i in range(num_inserted):
    assert str(i) in c_filter
    assert str(i) in b_filter

total_queries = 0
false_queries_cuckoo = 0
false_queries_bloom = 0 

#now let's test a range of numbers not inserted in the filter to estimate false positive rate
for i in range(num_inserted + 1, 10*max_items):
    if str(i) in c_filter:
        false_queries_cuckoo += 1
    
    if str(i) in b_filter:
        false_queries_bloom += 1
    
    total_queries += 1

print('Cuckoo filter false positive rate is {:%}'.format(false_queries_cuckoo / total_queries))
print('Counting bloom filter false positive rate is {:%}'.format(false_queries_bloom / total_queries))

Cuckoo filter false positive rate is 0.000000%
Counting bloom filter false positive rate is 0.008889%


Using the process coded up above, we estimate the false positive rate for cuckoo and bloom filters of different sizes. 
The graph below shows the empirical false positive rates derived from out tests. The code for this test as well as other tests can be found in the bench marking notebook provided in the repo. 

<img src="images/false_positive.png">

As seen from the image, we see that the cuckoo filter maintains an 5-8 time false positive rate advantage over the counting bloom filter. While our estimation procedure is not particularly exhaustive, the false positive rate test, and the other comparison (see bench marking notebook), indicate that the cuckoo filter, under the right circumstances, provides better performance than counting bloom filters. 

While the counting bloom filter can be further optimized and tuned to use less space and provide lower false positives, these optimizations could incure significant time and tuning. Here, our basic cuckoo filter implementation is already showing fairly superior performance across a whole host of metrics. 

### Real world application

While probabilistic data structures can seem esoteric, they are actually quite useful in practice. Consider the issue of engagement and growth that is typically of concern to large scale internet applications. For example, one common critic of Twitter is that it is difficult for new users to become engaged. To tackle this issue, members of the growth & engagement team at twitter can develop several marketting campaigns targetted at new and unengaged users in order to get them using twitter more often. To aid such work, every new user can be added to a cuckoo filter on registration. Now, once a user becomes active (whatever the definition of active is), the user can be removed from the cuckoo filter.  Consequently, the team can target growth campaigns to individuals currently in the cuckoo filter. From time to time, currently active users can lapse to become inactive. These users can be added and removed from the cuckoo filter depending on their activity level. Given the ease with which a cuckoo filter can be implemented, it is particularly amenable to such use case. For an application with 100s of millions of users, the cuckoo at its simplest implementation can help to provide a low memory footprint coupled with low false positive rates in this case. 

## Conclusion

Bloom filters and its variants have proven useful in streaming applications and others where membership testing is critical. In this post, we have shown how a cuckoo filter, which can be implemented simply provides better practical performance out of the box without tuning than counting bloom filters. Ultimately, cuckoo filters can serve as alternatives in scenarios where a counting bloom filter would normally be used. 